# OmicSelector2: Signature Benchmarking

This notebook demonstrates the core philosophy of OmicSelector: **automated benchmarking** of biomarker signatures.

**The OmicSelector Philosophy:**
Instead of trusting a single feature selection method, we:
1. Generate signatures with multiple methods
2. Test each signature with various ML models
3. Compare performance objectively
4. Select the signature most **resilient to overfitting**

**Learning Objectives:**
- Create multiple feature signatures
- Benchmark signatures across different models
- Compare signature performance statistically
- Identify the most robust biomarker set

**Prerequisites:**
```bash
pip install omicselector2
```

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Import OmicSelector2 components
from omicselector2.features.classical.lasso import LassoSelector
from omicselector2.features.classical.elastic_net import ElasticNetSelector
from omicselector2.features.classical.random_forest import RandomForestSelector
from omicselector2.features.classical.mrmr import MRMRSelector
from omicselector2.features.classical.boruta import BorutaSelector
from omicselector2.features.ensemble import EnsembleSelector
from omicselector2.features.stability import StabilitySelector

from omicselector2.models.classical import (
    RandomForestClassifier,
    XGBoostClassifier,
    LogisticRegressionModel
)
from omicselector2.training.benchmarking import SignatureBenchmark, Benchmarker
from omicselector2.training.cross_validation import StratifiedKFoldSplitter

# Set random seed
np.random.seed(42)

# Configure visualization
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Generate Dataset with Known Ground Truth

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Generate dataset with 20 truly informative genes
X, y = make_classification(
    n_samples=400,
    n_features=500,
    n_informative=20,
    n_redundant=30,
    n_repeated=0,
    n_classes=2,
    flip_y=0.02,
    class_sep=0.8,
    random_state=42
)

# Convert to DataFrame
X = pd.DataFrame(X, columns=[f"GENE_{i:03d}" for i in range(X.shape[1])])
y = pd.Series(y, name="response")

# Store ground truth informative genes
ground_truth_genes = [f"GENE_{i:03d}" for i in range(20)]

# Split into train and test (hold-out test set)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print(f"Dataset: {X.shape}")
print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Ground truth informative genes: {len(ground_truth_genes)}")

## 2. Create Multiple Feature Signatures

Generate biomarker signatures using different feature selection methods.

In [ ]:
# Dictionary to store signatures
signatures = {}

# Signature 1: Lasso (top 30 features)
lasso = LassoSelector(alpha=0.01, task='classification', n_features_to_select=30, random_state=42)
lasso.fit(X_train, y_train)
signatures['Lasso_30'] = lasso.selected_features_

# Signature 2: Elastic Net (top 30 features)
elastic_net = ElasticNetSelector(
    alpha=0.01, l1_ratio=0.7, task='classification', 
    n_features_to_select=30, random_state=42
)
elastic_net.fit(X_train, y_train)
signatures['ElasticNet_30'] = elastic_net.selected_features_

# Signature 3: Random Forest (top 30 features)
rf = RandomForestSelector(
    n_estimators=100, task='classification',
    n_features_to_select=30, random_state=42
)
rf.fit(X_train, y_train)
signatures['RandomForest_30'] = rf.selected_features_

# Signature 4: mRMR (top 30 features)
mrmr = MRMRSelector(n_features_to_select=30, random_state=42)
mrmr.fit(X_train, y_train)
signatures['mRMR_30'] = mrmr.selected_features_

# Signature 5: Boruta (all relevant features)
boruta = BorutaSelector(task='classification', max_iter=50, random_state=42)
boruta.fit(X_train, y_train)
signatures[f'Boruta_{len(boruta.selected_features_)}'] = boruta.selected_features_

# Signature 6: Ensemble (consensus of top 3 methods)
ensemble = EnsembleSelector(
    selectors=[lasso, elastic_net, rf],
    strategy='soft_voting',
    n_features_to_select=25
)
ensemble.fit(X_train, y_train)
signatures['Ensemble_25'] = ensemble.selected_features_

# Signature 7: Stability selection (robust features)
stability = StabilitySelector(
    base_selector=RandomForestSelector(n_estimators=50, task='classification'),
    n_bootstraps=50,
    threshold=0.6,
    sample_fraction=0.8,
    n_features_to_select=25,
    random_state=42
)
stability.fit(X_train, y_train)
signatures['Stability_25'] = stability.selected_features_

# Display signatures
print("\nGenerated Signatures:")
for name, features in signatures.items():
    # Count overlap with ground truth
    overlap = len(set(features) & set(ground_truth_genes))
    print(f"  {name}: {len(features)} features ({overlap}/{len(ground_truth_genes)} ground truth)")

## 3. Benchmark Each Signature

Test each signature with multiple ML models using cross-validation.

In [ ]:
# Define models to test
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBoostClassifier(n_estimators=100, random_state=42),
    'LogisticRegression': LogisticRegressionModel(random_state=42)
}

# Create benchmarker
benchmarker = Benchmarker(
    models=models,
    cv_strategy=StratifiedKFoldSplitter(n_splits=5, shuffle=True, random_state=42),
    metrics=['accuracy', 'auc', 'f1', 'precision', 'recall']
)

# Benchmark all signatures
print("Benchmarking signatures (this may take a few minutes)...\n")

benchmark_results = benchmarker.benchmark_signatures(
    X_train, y_train, signatures, verbose=True
)

print("\nBenchmarking complete!")

## 4. Compare Signature Performance

Visualize and compare cross-validation performance.

In [ ]:
# Get summary table
summary_df = benchmarker.get_summary_table()

print("\nBenchmark Summary (CV Performance):")
print(summary_df.to_string())

# Save to CSV
summary_df.to_csv('signature_benchmark_results.csv', index=False)
print("\nResults saved to 'signature_benchmark_results.csv'")

In [ ]:
# Visualize AUC comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: AUC by signature and model
auc_data = summary_df.pivot(index='signature', columns='model', values='cv_auc')
auc_data.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_ylabel('AUC-ROC')
axes[0].set_xlabel('Signature')
axes[0].set_title('Cross-Validation AUC by Signature and Model')
axes[0].legend(title='Model')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0.5, 1.0])

# Plot 2: Mean AUC per signature (across all models)
mean_auc_per_sig = summary_df.groupby('signature')['cv_auc'].mean().sort_values(ascending=False)
mean_auc_per_sig.plot(kind='barh', ax=axes[1], color='steelblue')
axes[1].set_xlabel('Mean AUC (across all models)')
axes[1].set_ylabel('Signature')
axes[1].set_title('Average Performance Across Models')
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim([0.5, 1.0])

plt.tight_layout()
plt.show()

print("\nBest signatures (by mean AUC):")
for sig, auc in mean_auc_per_sig.head(3).items():
    print(f"  {sig}: {auc:.4f}")

## 5. Statistical Comparison

Perform statistical tests to determine if performance differences are significant.

In [ ]:
# Compare top 2 signatures statistically
top_2_signatures = mean_auc_per_sig.head(2).index.tolist()
sig1_name, sig2_name = top_2_signatures

print(f"\nStatistical Comparison: {sig1_name} vs {sig2_name}")
print("=" * 60)

# Get detailed results for both signatures
sig1_results = benchmarker.results[sig1_name]
sig2_results = benchmarker.results[sig2_name]

# Compare across all models
for model_name in models.keys():
    sig1_aucs = sig1_results['fold_metrics'][model_name]['auc']
    sig2_aucs = sig2_results['fold_metrics'][model_name]['auc']
    
    # Paired t-test (same folds)
    t_stat, p_value = stats.ttest_rel(sig1_aucs, sig2_aucs)
    
    print(f"\n{model_name}:")
    print(f"  {sig1_name}: {np.mean(sig1_aucs):.4f} ± {np.std(sig1_aucs):.4f}")
    print(f"  {sig2_name}: {np.mean(sig2_aucs):.4f} ± {np.std(sig2_aucs):.4f}")
    print(f"  Paired t-test: t={t_stat:.3f}, p={p_value:.4f}")
    
    if p_value < 0.05:
        winner = sig1_name if np.mean(sig1_aucs) > np.mean(sig2_aucs) else sig2_name
        print(f"  → {winner} is significantly better (p < 0.05)")
    else:
        print(f"  → No significant difference (p >= 0.05)")

## 6. Test Set Evaluation

Evaluate the best signature on the held-out test set.

In [ ]:
# Get best signature
best_signature_name = mean_auc_per_sig.idxmax()
best_signature = signatures[best_signature_name]

print(f"Best Signature: {best_signature_name}")
print(f"Number of features: {len(best_signature)}")
print(f"\nFeatures:")
for i, gene in enumerate(best_signature[:20], 1):
    is_ground_truth = '✓' if gene in ground_truth_genes else ''
    print(f"  {i}. {gene} {is_ground_truth}")

if len(best_signature) > 20:
    print(f"  ... and {len(best_signature) - 20} more")

In [ ]:
# Train final model on best signature
X_train_best = X_train[best_signature]
X_test_best = X_test[best_signature]

# Try all three models
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

print("\nTest Set Performance (Hold-out Validation):")
print("=" * 60)

test_results = {}

for model_name, model in models.items():
    # Train on full training set
    model.fit(X_train_best, y_train)
    
    # Predict on test set
    y_pred = model.predict(X_test_best)
    y_pred_proba = model.predict_proba(X_test_best)[:, 1]
    
    # Calculate metrics
    test_auc = roc_auc_score(y_test, y_pred_proba)
    test_results[model_name] = {
        'auc': test_auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"\n{model_name}:")
    print(f"  Test AUC: {test_auc:.4f}")
    
# Find best model on test set
best_test_model = max(test_results, key=lambda k: test_results[k]['auc'])
print(f"\n→ Best model on test set: {best_test_model} (AUC = {test_results[best_test_model]['auc']:.4f})")

In [ ]:
# Plot ROC curves for all models
plt.figure(figsize=(10, 8))

for model_name, results in test_results.items():
    fpr, tpr, _ = roc_curve(y_test, results['y_pred_proba'])
    plt.plot(fpr, tpr, label=f"{model_name} (AUC = {results['auc']:.3f})", linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title(f'ROC Curves - {best_signature_name} (Test Set)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Signature Quality Analysis

Analyze the quality of the best signature.

In [ ]:
# Calculate overlap with ground truth
overlap_genes = set(best_signature) & set(ground_truth_genes)
false_positives = set(best_signature) - set(ground_truth_genes)

print(f"Signature Quality Analysis: {best_signature_name}")
print("=" * 60)
print(f"Total features: {len(best_signature)}")
print(f"True positives (overlap with ground truth): {len(overlap_genes)}/{len(ground_truth_genes)} ({len(overlap_genes)/len(ground_truth_genes)*100:.1f}%)")
print(f"False positives (noise features): {len(false_positives)}")
print(f"Precision: {len(overlap_genes)/len(best_signature)*100:.1f}%")
print(f"\nTrue positive genes:")
for gene in sorted(overlap_genes):
    print(f"  ✓ {gene}")

## Summary & Recommendations

### The OmicSelector Workflow:

1. **Generate Multiple Signatures** - Use diverse feature selection methods
2. **Benchmark Systematically** - Test with multiple ML models and CV
3. **Compare Objectively** - Use statistics to identify truly better signatures
4. **Validate Rigorously** - Hold-out test set evaluation
5. **Select Most Robust** - Choose signature resilient to overfitting

### Key Findings from This Analysis:

- **Best Signature:** The signature that performed best across all models
- **Stability Matters:** Signatures selected by ensemble/stability methods often generalize better
- **Model Agnostic:** Good signatures perform well across different ML algorithms
- **Ground Truth Recovery:** We successfully recovered a substantial portion of informative features

### Recommendations for Real Biomarker Discovery:

1. Always use **hold-out validation** (or independent cohort validation)
2. Test signatures with **multiple models** - performance should be consistent
3. Prefer **ensemble/stability methods** for robust feature selection
4. Use **statistical tests** to confirm differences are not due to chance
5. Consider **biological validation** (pathway enrichment, literature review)

### Next Steps:

- Apply this workflow to your real biomarker data
- Experiment with different signature sizes (10, 20, 50, 100 features)
- Try additional feature selection methods
- Validate top signatures in independent cohorts
- Perform pathway enrichment analysis on selected genes